<p><strong><span style="font-size: 24pt;">Loading the dataset + Splitting it to train and test</span></strong></p>

In [ ]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import least_squares,minimize
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import least_squares,minimize
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the data from the CSV file
data = pd.read_csv('PDE_data.csv')

# Separate input (X) and output (Y)
inputs = data.iloc[:, :5].values
outputs = data.iloc[:, 5:].values

# Initialize MinMaxScaler for inputs
input_scaler = MinMaxScaler()
scaled_inputs = input_scaler.fit_transform(inputs)
# Convert the scaled data to a PyTorch tensor
scaled_inputs_tensor = torch.tensor(scaled_inputs, dtype=torch.float32)
# Split the output data into 4 signals
signal1 = outputs[:, :32]
signal2 = outputs[:, 32:64]
signal3 = outputs[:, 64:96]
signal4 = outputs[:, 96:]

# Initialize MinMaxScaler for outputs
output_scaler = MinMaxScaler()

# Scale each signal individually to the range of 0-1
scaled_signal1 = output_scaler.fit_transform(signal1)
scaled_signal2 = output_scaler.fit_transform(signal2)
scaled_signal3 = output_scaler.fit_transform(signal3)
scaled_signal4 = output_scaler.fit_transform(signal4)

# Combine the scaled signals into a new output with 128 components
scaled_outputs = np.concatenate((scaled_signal1, scaled_signal2, scaled_signal3, scaled_signal4), axis=1)

# Convert the scaled output data to a PyTorch tensor
scaled_outputs_tensor = torch.tensor(scaled_outputs, dtype=torch.float32)

# Split data into training and testing sets (80-20 split)
train_X, test_X, train_Y, test_Y = train_test_split(scaled_inputs_tensor, scaled_outputs, test_size=0.2, random_state=42)

# Convert the input and output data to PyTorch tensors
train_x = torch.tensor(train_X, dtype=torch.float32)
test_x = torch.tensor(test_X, dtype=torch.float32)
train_y = torch.tensor(train_Y, dtype=torch.float32)
test_y = torch.tensor(test_Y, dtype=torch.float32)


<p><strong><span style="font-size: 24pt;">Applying PCA to reduce outputs diminsion</span></strong></p>

In [7]:
train_Y_reshaped = train_Y.reshape(-1, train_Y.shape[1])
test_Y_reshaped = test_Y.reshape(-1, test_Y.shape[1])
# Specify batch size
batch_size = 50

# Create IncrementalPCA object
pca = IncrementalPCA(n_components=12)

# Fit the model in batches
for i in range(0, len(train_Y_reshaped), batch_size):
    batch = train_Y_reshaped[i:i + batch_size, :]
    pca.partial_fit(batch)

# Transform training and testing data to reduced dimensions
train_y_pca = pca.transform(train_Y_reshaped)
test_y_pca = pca.transform(test_Y_reshaped)

<p><strong><span style="font-size: 24pt;">Creating the GP model ( multitask)</span></strong></p>

In [8]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_Y_pca, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_Y_pca, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=12
        )
        
        # Using Matern kernel instead of RBFKernel
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.MaternKernel(), num_tasks=12, rank=0
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=12)
model = MultitaskGPModel(train_x, torch.Tensor(train_y_pca), likelihood)


In [5]:
import os
smoke_test = ('CI' in os.environ)
training_iterations = 2 if smoke_test else 1000


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, torch.Tensor(train_y_pca))
    loss.backward()
    if (i%50==0):
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    
    optimizer.step()
    

Iter 1/1000 - Loss: 1.121
Iter 51/1000 - Loss: -0.983
Iter 101/1000 - Loss: -1.743
Iter 151/1000 - Loss: -1.811
Iter 201/1000 - Loss: -1.826
Iter 251/1000 - Loss: -1.832
Iter 301/1000 - Loss: -1.836
Iter 351/1000 - Loss: -1.839
Iter 401/1000 - Loss: -1.841
Iter 451/1000 - Loss: -1.842
Iter 501/1000 - Loss: -1.843
Iter 551/1000 - Loss: -1.844
Iter 601/1000 - Loss: -1.845
Iter 651/1000 - Loss: -1.845
Iter 701/1000 - Loss: -1.845
Iter 751/1000 - Loss: -1.846
Iter 801/1000 - Loss: -1.846
Iter 851/1000 - Loss: -1.847
Iter 901/1000 - Loss: -1.847
Iter 951/1000 - Loss: -1.847


<h2 id="Importing-true-DATA"><strong><span style="font-size: 24pt;">Importing TRUE measured DATA</span></strong></h2>

In [10]:
# Load the measured_data from the Excel file
data_m = pd.read_csv('MEASURED_DATA.csv')
#true_inputs_scaled = scaler.transform(new_inputs_tensor).flatten()
true_outputs = data_m.iloc[:, :].values
true_outputs=torch.Tensor(true_outputs)
true_pressure_m = true_outputs[:,:2].cpu().numpy().flatten()
true_flow1_m = true_outputs[:,2:34].cpu().numpy().flatten()
true_flow2_m = true_outputs[:,34:66].cpu().numpy().flatten()
true_area_m = true_outputs[:,66:98].cpu().numpy().flatten()

<p><strong><span style="font-size: 24pt;">Comparing GP predictions with True test Data</span></strong></p>

In [ ]:
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Set the model and likelihood to evaluation mode
model.eval()
likelihood.eval()
num_samples = 5  # Number of samples to consider from test data

# Make predictions on the test set using the model trained on the training data
# MJC: Since GPs naturally incorporate uncertainty, I've added the plotting commands for that here
with torch.no_grad(), gpytorch.settings.fast_computations():
    model.eval()  # Set the model to evaluation mode
    q = torch.Tensor(np.array(test_x[:num_samples], ndmin=2))  # Select the first 10 samples
    predictions = model(q)
    lower, upper = predictions.confidence_region()

# Extract the means of the predictions for each output task
means_pred_subset = predictions.mean.detach().numpy()

# Inverse transform the predictions back to the original dimensionality of 32 features
model_predictions_original_dim = pca.inverse_transform(means_pred_subset)

# Define MinMaxScaler for each signal individually
output_scaler1 = MinMaxScaler()
output_scaler2 = MinMaxScaler()
output_scaler3 = MinMaxScaler()
output_scaler4 = MinMaxScaler()

# Fit and transform each signal individually
scaled_signal1 = output_scaler1.fit_transform(signal1)
scaled_signal2 = output_scaler2.fit_transform(signal2)
scaled_signal3 = output_scaler3.fit_transform(signal3)
scaled_signal4 = output_scaler4.fit_transform(signal4)

# Use the transformed predictions to calculate the confidence intervals
lower_original_dim, upper_original_dim = pca.inverse_transform(lower.numpy()), pca.inverse_transform(upper.numpy())

# Inverse transform the predictions back to the original scale
original_dim_predictions = np.zeros_like(model_predictions_original_dim)

# Loop over each signal and perform inverse transformation
for i in range(4):
    if i == 0:
        original_dim_predictions[:, i*32:(i+1)*32] = output_scaler1.inverse_transform(model_predictions_original_dim[:, i*32:(i+1)*32])
    elif i == 1:
        original_dim_predictions[:, i*32:(i+1)*32] = output_scaler2.inverse_transform(model_predictions_original_dim[:, i*32:(i+1)*32])
    elif i == 2:
        original_dim_predictions[:, i*32:(i+1)*32] = output_scaler3.inverse_transform(model_predictions_original_dim[:, i*32:(i+1)*32])
    else:
        original_dim_predictions[:, i*32:(i+1)*32] = output_scaler4.inverse_transform(model_predictions_original_dim[:, i*32:(i+1)*32])

# Inverse transform the actual test data back to the original scale, using different scalers for each signal
selected_test_y_np = test_y[:num_samples].cpu().numpy()
original_dim_test_y = np.zeros_like(selected_test_y_np)

for i in range(4):
    if i == 0:
        original_dim_test_y[:, i*32:(i+1)*32] = output_scaler1.inverse_transform(selected_test_y_np[:, i*32:(i+1)*32])
    elif i == 1:
        original_dim_test_y[:, i*32:(i+1)*32] = output_scaler2.inverse_transform(selected_test_y_np[:, i*32:(i+1)*32])
    elif i == 2:
        original_dim_test_y[:, i*32:(i+1)*32] = output_scaler3.inverse_transform(selected_test_y_np[:, i*32:(i+1)*32])
    else:
        original_dim_test_y[:, i*32:(i+1)*32] = output_scaler4.inverse_transform(selected_test_y_np[:, i*32:(i+1)*32])

# Define the y-axis titles for each signal
y_axis_titles = ['MPA Pressure (mmHg)', 'LPA Flow (mL/s)', 'RPA Flow (mL/s)', 'MPA area (cm2)']

# Loop over each sample
for j in range(num_samples):
    plt.figure(figsize=(15, 6))
    
    # Initialize lists to store differences for each signal
    differences = []
    
    # Loop over each signal
    for i in range(4):
        plt.subplot(1, 4, i+1)
        
        # Plot model prediction
        plt.plot(np.arange(32), original_dim_predictions[j, i*32:(i+1)*32], color='blue', alpha=0.5, label='GP')
        
        # Plot actual data as a curve
        plt.plot(np.arange(32), original_dim_test_y[j, i*32:(i+1)*32], color='red', label='transfomed back from scaling')
        
        # Plot test_yy data as black x markers
        plt.plot(np.arange(32), test_yy[j, i*32:(i+1)*32], 'kx', label='original signal')
        
        plt.ylabel(y_axis_titles[i])
        plt.title(y_axis_titles[i])
        plt.legend()
    
    plt.tight_layout()
    plt.show()

<h4 id="OPTIMIZATION"><span style="font-size: 24pt;">OPTIMIZATION ( It's good to do OPT first. so we can have a good intial values for MCMC)&nbsp;<br /></span></h4>

In [17]:
# Define the function before using it in the optimization loop
def OPT_SSE_realdata(q, true_pressure_data, true_flow1, true_flow2, true_area):
    n_pts = 32

    model.eval()  # Set the model to evaluation mode
    q = torch.Tensor([q])
    predictions = model(q)
    model_predictions = predictions.mean.detach().numpy().flatten()
    model_predictions = pca.inverse_transform(model_predictions)

    # Define MinMaxScaler for each signal individually
    output_scaler1 = MinMaxScaler()
    output_scaler2 = MinMaxScaler()
    output_scaler3 = MinMaxScaler()
    output_scaler4 = MinMaxScaler()

    # Fit and transform each signal individually
    scaled_signal1 = output_scaler1.fit_transform(signal1)
    scaled_signal2 = output_scaler2.fit_transform(signal2)
    scaled_signal3 = output_scaler3.fit_transform(signal3)
    scaled_signal4 = output_scaler4.fit_transform(signal4)

    # Inverse transform each signal back to the original space
    model_predictions_original_dim = np.zeros((1, 128))  # Initialize array with correct dimensions
    model_predictions_original_dim[:, :32] = output_scaler1.inverse_transform(model_predictions[:32].reshape(1, -1))
    model_predictions_original_dim[:, 32:64] = output_scaler2.inverse_transform(model_predictions[32:64].reshape(1, -1))
    model_predictions_original_dim[:, 64:96] = output_scaler3.inverse_transform(model_predictions[64:96].reshape(1, -1))
    model_predictions_original_dim[:, 96:] = output_scaler4.inverse_transform(model_predictions[96:].reshape(1, -1))

    # Extract pressure data from the first 32 components of model_predictions
    p_predictions = model_predictions_original_dim[:, :32].flatten()

    # Create a parameter containing only the max and min of the first 32 components
    max_min_pressure_model = np.array([np.max(p_predictions), np.min(p_predictions)])

    # Calculate MSE for each subgroup
    mse_pressure = np.sum((true_pressure_data - max_min_pressure_model)**2)
    mse_flow1 = np.sum((true_flow1 - model_predictions_original_dim[:, 32:64].flatten())**2)
    mse_flow2 = np.sum((true_flow2 - model_predictions_original_dim[:, 64:96].flatten())**2)
    mse_area = np.sum((true_area - model_predictions_original_dim[:, 96:].flatten())**2)
    total_mse = mse_pressure + mse_flow1 + mse_flow2 + mse_area
    print(mse_pressure,mse_flow1,mse_flow2,mse_area)

    return total_mse

In [ ]:
import random
# Define which dataset to use
true_pressure_data = true_pressure_m.flatten()
true_flow1_data = true_flow1_m.flatten()
true_flow2_data = true_flow2_m.flatten()
true_area = true_area_m.flatten()
#print(true_pressure_data)
# Initial guess
# Define the bounds for the optimization
X0_true = torch.Tensor([[0.24, 0.2, 0.98, 0.44, 0.51]])
total_costs = []
param_save = []
bounds = [(0, 1.0), (0, 1.0), (0, 1.0), (0, 1.0), (0,1.0)]  

for i in range(8):
    # Generate random perturbation within ±10% of X0_2
    perturbation = torch.rand(X0_true.size()) * 0.1 - 0.05  # ±10% range
    
    # Apply perturbation to create a new guess
    new_guess = X0_true * (1 + perturbation)
    
    # Perform optimization with new_guess
    results = minimize(OPT_SSE_realdata, new_guess, args=(true_pressure_data, true_flow1_data, true_flow2_data, true_area),
                       method='powell', options={'disp': False, 'maxiter': 1000, 'gtol': 1e-6, 'maxcor': 10})
    
    # Calculate total_cost based on the optimization results 
    total_cost = results.fun
    
    # Append the total_cost to the list
    total_costs.append(total_cost)
    param_save.append(results.x)
    
    # Print indication for the start of the next iteration
    if i < 7:
        print(f"Next iteration ({i+2}/8) starting...")

# Print all total_costs
print("Total costs for each iteration:", total_costs)
print(total_costs)
print(param_save)

## MCMC for REAL DATA

In [ ]:
import pymcmcstat
from scipy.integrate import odeint, solve_ivp
from scipy.optimize import least_squares,minimize
import matplotlib.pyplot as plt
from pymcmcstat.MCMC import MCMC # This is the MCMC package we use
from pymcmcstat.settings.DataStructure import DataStructure # This is the sub module that is useful for MCMC
from dataclasses import dataclass
import ast
from pymcmcstat import MCMC
import pymcmcstat
from pymcmcstat.MCMC import MCMC
import pickle
import matplotlib.pyplot as plt
from pymcmcstat import mcmcplot
from scipy.stats import gaussian_kde

In [ ]:
def SSE_realdata(q, data):
    n_pts = 32

    model.eval()  # Set the model to evaluation mode
    q = torch.Tensor([q])
    predictions = model(q)
    model_predictions = predictions.mean.detach().numpy().flatten()
    model_predictions = pca.inverse_transform(model_predictions)

    # Define MinMaxScaler for each signal individually
    output_scaler1 = MinMaxScaler()
    output_scaler2 = MinMaxScaler()
    output_scaler3 = MinMaxScaler()
    output_scaler4 = MinMaxScaler()

    # Fit and transform each signal individually
    scaled_signal1 = output_scaler1.fit_transform(signal1)
    scaled_signal2 = output_scaler2.fit_transform(signal2)
    scaled_signal3 = output_scaler3.fit_transform(signal3)
    scaled_signal4 = output_scaler4.fit_transform(signal4)

    # Inverse transform each signal back to the original space
    model_predictions_original_dim = np.zeros((1, 128))  # Initialize array with correct dimensions
    model_predictions_original_dim[:, :32] = output_scaler1.inverse_transform(model_predictions[:32].reshape(1, -1))
    model_predictions_original_dim[:, 32:64] = output_scaler2.inverse_transform(model_predictions[32:64].reshape(1, -1))
    model_predictions_original_dim[:, 64:96] = output_scaler3.inverse_transform(model_predictions[64:96].reshape(1, -1))
    model_predictions_original_dim[:, 96:] = output_scaler4.inverse_transform(model_predictions[96:].reshape(1, -1))

    # Extract pressure data from the first 32 components of model_predictions
    p_predictions = model_predictions_original_dim[:, :32].flatten()

    # Create a parameter containing only the max and min of the first 32 components
    max_min_pressure_model = np.array([np.max(p_predictions), np.min(p_predictions)])

    # Separate true data into subgroups
    true_pressure_data = data.ydata[0].flatten()
    true_max_min_p = np.array([(np.max(true_pressure_data), np.min(true_pressure_data))], dtype=type(true_pressure_data))
    true_flow1_data = data.ydata[1].flatten()
    true_flow2_data = data.ydata[2].flatten()
    true_area_data = data.ydata[3].flatten()

    # Calculate MSE for each subgroup
    mse_pressure = np.sum((true_max_min_p - max_min_pressure_model)**2)
    mse_flow1 = np.sum((true_flow1_data - model_predictions_original_dim[:, 32:64].flatten())**2)
    mse_flow2 = np.sum((true_flow2_data - model_predictions_original_dim[:, 64:96].flatten())**2)
    mse_area = np.sum((true_area_data - model_predictions_original_dim[:, 96:].flatten())**2)
    total_mse = [mse_pressure, mse_flow1, mse_flow2, mse_area]

    total_mse = np.transpose([mse_pressure, mse_flow1, mse_flow2, mse_area])

    return total_mse

# Full parameter set
theta0 = dict(k = 0.14, gamma1 = 0.5, lrr1=0.68, gamma2 = 0.8, lrr2=0.51) # initial values which are inputes_meas_scaled
theta0vec = list(theta0.values())
bounds = dict(k = [0,1], gamma1 = [0,1], lrr1 = [0,1], gamma2 = [0,1], lrr2 = [0,1])
names = ['k','gamma1','lrr1','gamma2','lrr2']
longnames = ['$k$','$gamma1$','$lrr1$','$gamma2$','$lrr2$']

data = DataStructure()

## NOTE: using a different name here so that we don't overwrite results
p_data = true_pressure_m
q1_data = true_flow1_m
q2_data = true_flow2_m
area_data = true_area_m


mcstat_pluto = MCMC()
mcstat_pluto.data.add_data_set(np.linspace(0,1,2),p_data)
mcstat_pluto.data.add_data_set(np.linspace(0,1,32),q1_data)
mcstat_pluto.data.add_data_set(np.linspace(0,1,32),q2_data)
mcstat_pluto.data.add_data_set(np.linspace(0,1,32),area_data)
mse0 = MJC_SSE_realdata([0.14, 0.5, 0.68, 0.8, 0.51],mcstat_pluto.data) 
print(mse0)

for ii, name in enumerate(names):
    mcstat_pluto.parameters.add_model_parameter(
    name = longnames[ii],
    theta0=theta0[name],
    minimum=bounds[name][0],
    maximum=bounds[name][1])
    
mcstat_pluto.model_settings.define_model_settings(sos_function=SSE_realdata,N = np.array([2,32,32,32]))
       
mcstat_pluto.simulation_options.define_simulation_options(
        nsimu=25000,
        method='dram', # Options are 'mh' for metropolis hastings, 'am' for adaptive metroplois, and 'dram'
        waitbar=True,
        updatesigma=True) # Always set to true
with torch.no_grad(), gpytorch.settings.fast_computations():
    mcstat_pluto.run_simulation()
import pickle
# Calculate chain statistics
chainstats_results = mcstat_pluto.chainstats(mcstat_pluto.simulation_results.results['chain'][:, :],
                                              mcstat_pluto.simulation_results.results)
# Save relevant attributes
results_to_save = {
    'simulation_results': mcstat_pluto.simulation_results.results,
    'chainstats_results': chainstats_results,
    # Add any other necessary data
}

# Save the results
with open("Sample.pickle", "wb") as m:
    pickle.dump(results_to_save, m)

<p><strong><span style="font-size: 24pt;">Posterior density</span></strong></p>

In [ ]:
# Load saved results
with open("Sample.pickle", "rb") as f:
    saved_results = pickle.load(f)

# Extract relevant data
simulation_results = saved_results['simulation_results']
chainstats_results = saved_results.get('chainstats_results')  # Check if chainstats_results exist

# Assuming chain has shape (nsimu, npar) and names contains the parameter names
names = ['k', 'gamma1', 'lrr1', 'gamma2', 'lrr2']  # Modify with actual parameter names

# Plot density panel
fig = mcmcplot.plot_density_panel(simulation_results['chain'][:, :], names)

# Set the color of the density plot to blue
for ax in fig.axes:
    ax.get_lines()[0].set_color('red')

# Set x-axis limits to 0-1 for all subplots
for ax in fig.axes:
    ax.set_xlim(-0.1, 1.1)

# Calculate mode for each parameter and plot as vertical lines
for ax, param_name, chain in zip(fig.axes, names, simulation_results['chain'][:, :].T):
    kde = gaussian_kde(chain)
    x = np.linspace(min(chain), max(chain), 1000)
    density = kde(x)
    mode = x[np.argmax(density)]
    ax.axvline(mode, color='blue', linestyle='--', label=f'Mode: {mode:.2f}')
    ax.legend()

# Show the plot
plt.show()

<p><strong><span style="font-size: 24pt;">Correlations plots</span></strong></p>

In [ ]:
# Extract relevant data
results = saved_results['simulation_results']
chain = results['chain']
s2chain = results['s2chain']
names = results['names']

# Plot pairwise correlation panel
plt.figure
mcmcplot.plot_pairwise_correlation_panel(chain, names)

# Show the plot
plt.show()

<p><span style="font-size: 24pt;"><strong>Plotting the GP predictions vs True data using posteriors</strong></span></p>

In [ ]:
# Assuming 'X0_true' is mode valuse from the posterirors 
X0_true = torch.tensor([0.14, 0.01, 0.99, 0.52, 0.96], dtype=torch.float32) #[0.72, 0.42, 0.97, 0.05, 0.76

# Reshape the new input to match the expected size
X0_true = X0_true.view(1, -1)

# Set the model and likelihood to evaluation mode
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_computations():
    model.eval()  # Set the model to evaluation mode
    q = torch.Tensor(np.array(X0_true, ndmin=2))
    predictions = model(q)
    model_predictions = predictions.mean.detach().numpy().flatten()
    model_predictions = pca.inverse_transform(model_predictions)
output_scaler1 = MinMaxScaler()
output_scaler2 = MinMaxScaler()
output_scaler3 = MinMaxScaler()
output_scaler4 = MinMaxScaler()

# Fit and transform each signal individually
scaled_signal1 = output_scaler1.fit_transform(signal1)
scaled_signal2 = output_scaler2.fit_transform(signal2)
scaled_signal3 = output_scaler3.fit_transform(signal3)
scaled_signal4 = output_scaler4.fit_transform(signal4)

    # Inverse transform each signal back to the original space
model_predictions_original_dim = np.zeros((1, 128))  # Initialize array with correct dimensions
model_predictions_original_dim[:, :32] = output_scaler1.inverse_transform(model_predictions[:32].reshape(1, -1))
model_predictions_original_dim[:, 32:64] = output_scaler2.inverse_transform(model_predictions[32:64].reshape(1, -1))
model_predictions_original_dim[:, 64:96] = output_scaler3.inverse_transform(model_predictions[64:96].reshape(1, -1))
model_predictions_original_dim[:, 96:] = output_scaler4.inverse_transform(model_predictions[96:].reshape(1, -1))

# Actual measurements for MPA pressure (min, max) and LPA/RPA flow
true_MPA_pressure_min = 34.5  # actual sys pressure
true_MPA_pressure_max = 21  # actual dias pressure
true_LPA_flow = true_flow1_m  # actual LPA flow signal
true_RPA_flow = true_flow2_m
true_MPA_area= true_area_m 

## GP predictions
p_GP=model_predictions_original_dim[:, :32].flatten()
LPA_flow_GP=model_predictions_original_dim[:, 32:64].flatten()
RPA_flow_GP=model_predictions_original_dim[:, 64:96].flatten()
a_GP=model_predictions_original_dim[:, 96:].flatten()
# Plot the predictions for the single input along with actual measurements
plt.figure(figsize=(12, 6))
plt.subplot(141)
plt.plot(p_GP, color='blue', alpha=0.8, label='GP')
plt.axhline(y=true_MPA_pressure_min, color='red', linestyle='--',)
plt.axhline(y=true_MPA_pressure_max, color='red', linestyle='--', label='Measured')
plt.ylabel('MPA Pressure (mmHg)')
plt.title('MPA pressure')
plt.legend()
plt.xticks([0, 16, 32], ['0', 'T/2', 'T'])

plt.subplot(142)
plt.plot(LPA_flow_GP, color='blue', alpha=0.8, label='GP')
plt.plot(true_LPA_flow, color='red', linestyle='--', label='Measured')
plt.ylabel('LPA Flow (mL/s)')
plt.title('LPA Flow')
plt.legend()
plt.xticks([0, 16, 32], ['0', 'T/2', 'T'])

plt.subplot(143)
plt.plot(RPA_flow_GP, color='blue', alpha=0.8, label='GP')
plt.plot(true_RPA_flow, color='red', linestyle='--', label='Measured')
plt.ylabel('RPA Flow (mL/s)')
plt.title('RPA Flow')
plt.legend()
plt.xticks([0, 16, 32], ['0', 'T/2', 'T'])

plt.subplot(144)
plt.plot(a_GP, color='blue', alpha=0.8, label='GP')
plt.plot(np.roll(true_area_m,1), color='red', linestyle='--', label='Measured')
plt.ylabel('Area(cm2)')
plt.title('MPA area')
plt.legend()
plt.xticks([0, 16, 32], ['0', 'T/2', 'T'])

plt.tight_layout()
plt.show()